In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
import selenium
import pickle
import time
from datetime import datetime
import unicodedata
import random
import urllib.parse
# from seleniumwire import webdriver
from selenium.webdriver.common.proxy import *


In [47]:
f = pd.read_excel("/home/neafiol/Downloads/Telegram Desktop/url-yam.xlsx")

In [63]:
urls=[]
for i in list(f['Downloading: https://market.yandex.ru/sitemap/sitemap-model_reviews.xml']):
    if "Found" in i:
        urls.append(i[6:-8])
urls=list(set(urls))
urls=[{'url':u} for u in urls]

In [64]:
urls

[{'url': 'https://market.yandex.ru/product--gorodskoi-velosiped-corto-fb116/10822589'},
 {'url': 'https://market.yandex.ru/product--vino-dom-solntsa-tamianka-livadiia-0-7-l/593319002'},
 {'url': 'https://market.yandex.ru/product--konvektor-electrolux-ech-as-1500-er/1732136690'},
 {'url': 'https://market.yandex.ru/product--shvedskaia-stenka-romana-s4/562087034'},
 {'url': 'https://market.yandex.ru/product--videokarta-evga-geforce-gtx-650-ti-boost-1072mhz-pci-e-3-0-2048mb-6008mhz-192-bit-2xdvi-hdmi-hdcp/9340183'},
 {'url': 'https://market.yandex.ru/product--shchiptsy-bosch-phs2560/2447523'},
 {'url': 'https://market.yandex.ru/product--udlinitel-aopen-usb-usb-acu302-0-5-m/373284161'},
 {'url': 'https://market.yandex.ru/product--prorezyvatel-pogremushka-knopa-neptun/331143215'},
 {'url': 'https://market.yandex.ru/product--911-kids-krem-detskii-podorozhnik/1723169008'},
 {'url': 'https://market.yandex.ru/product--soska-mir-detstva-silikonovaia-klassicheskaia-reguliruemyi-potok-6m-2sht/17229

In [59]:
from peewee import MySQLDatabase, Model, IntegerField, TextField, BooleanField
from playhouse.postgres_ext import PostgresqlExtDatabase

from configs import *

db = PostgresqlExtDatabase(db_name, user=db_login, password=db_password,
                           host=db_host, port=db_port)


class Tasks(Model):
    url = TextField(unique=True)
    done = BooleanField(default=False)
    empty = BooleanField(default=False)

    class Meta:
        database = db


In [65]:
for i in tqdm(range(0,len(urls),1000)):
    Tasks.insert_many(urls[i:i+1000]).execute()

/home/neafiol/Documents/Projects/YaMarket/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [45]:
with open("urls.txt","w") as f:
    for u in urls:
        f.write(u['url']+"\n")

In [2]:
user_data_dir = "/home/neafiol/.config/google-selenium-chrome"

options = webdriver.ChromeOptions()
options.add_argument(f"--user-data-dir={user_data_dir}")
driver = webdriver.Chrome(options=options)


driver.get("https://market.yandex.ru")

In [8]:
driver.get("https://market.yandex.by/product--detraleks-tab-p-o-plen-1000-mg-60/186858138")

In [3]:
driver.find_element_by_id("header-search").send_keys("Молоко простокваштно",Keys.ENTER)

In [42]:
bool([])

False

In [53]:
page  = driver.page_source 
soup = BeautifulSoup(page, 'html5lib')

In [54]:
def get_options(soup):
    res = []
    for a in soup.find_all("article"):
        url = a.find("a", {"href": lambda x: x and "product" in x})
        if url:
            res.append({
                "name": a.find("a", {"title": True})["title"],
                "url": "https://market.yandex.ru" +
                      url["href"].split("?")[0],
                "price": a.find("span", {"data-autotest-currency": "₽"})["data-autotest-value"] if a.find("span", {
                    "data-autotest-currency": "₽"}) else 0
            })
    return res

In [55]:
opt = get_options(soup)
opt

[{'name': 'Сыр Coeur du nord бюш мягкий с белой плесенью в лепестках лука 45%',
  'url': 'https://market.yandex.ru/product--syr-coeur-du-nord-biush-miagkii-s-beloi-pleseniu-v-lepestkakh-luka-45/440082238',
  'price': 0},
 {'name': 'Сыр Coeur du nord бюш мягкий с белой плесенью 45%',
  'url': 'https://market.yandex.ru/product--syr-coeur-du-nord-biush-miagkii-s-beloi-pleseniu-45/440078247',
  'price': 0},
 {'name': 'Сыр Coeur du nord бюш мягкий с белой плесенью в лепестках роз 45%',
  'url': 'https://market.yandex.ru/product--syr-coeur-du-nord-biush-miagkii-s-beloi-pleseniu-v-lepestkakh-roz-45/440080062',
  'price': 0},
 {'name': 'Сыр DORBLU Сыр с голубой плесенью a la creme 61%',
  'url': 'https://market.yandex.ru/product--syr-dorblu-syr-s-goluboi-pleseniu-a-la-creme-61/546432018',
  'price': '146'},
 {'name': 'Сыр Coeur du nord камамбер мягкий с белой плесенью 45%',
  'url': 'https://market.yandex.ru/product--syr-coeur-du-nord-kamamber-miagkii-s-beloi-pleseniu-45/440078045',
  'price':

In [33]:
def get_params(soup):
        params = {}
        for div in soup.find("div", {"data-apiary-widget-id": "/content/productSpecs"}).div.div.find_all("div"):
            if len(div.find_all("dl")) > 0:
                for dl in div.find_all("dl"):
                    params[dl.dt.span.text] = dl.dd.text
        if len(params) == 0:
            raise Exception("No params found")
        return params

In [29]:
driver.get(opt[0]["url"]+"/spec")

In [34]:
page  = driver.page_source 
soup = BeautifulSoup(page, 'html5lib')
get_params(soup)

{'Тип молока': 'коровье',
 'Способ обработки': 'ультрапастеризация',
 'Жирность': '3.2 %',
 'Объем': '0.95 л',
 'Не содержит': 'консерванты',
 'Упаковка': 'тетра-пак',
 'С крышечкой': 'да',
 'Стандарты производства': 'ТУ 10.51.11-050-13605199',
 'Энергетическая ценность в 100 г': '59 ккал',
 'Белки в 100 г': '2.9 г',
 'Жиры в 100 г': '3.2 г',
 'Углеводы в 100 г': '4.7 г',
 'Состав': 'нормализованное молоко',
 'Срок годности': '9 мес.'}

## Session

In [25]:
def get_main(soup):
    info = {}
    info["brand"] = soup.find("meta", {"itemprop": "brand"})["content"]
    info["category"] = soup.find("meta", {"itemprop": "category"})["content"]
    info["name"] = soup.find("meta", {"itemprop": "name"})["content"]
    info["cat_url"] = soup.find("meta", {"itemprop": "url","content":lambda x: "product" in x})["content"]
    info["description"] = soup.find("meta", {"itemprop": "description"})["content"]
    info["price"] = soup.find("meta", {"itemprop": "lowPrice"})["content"]


    info["imgs"] = [i["src"].replace("1hq", "10hq").replace("//", "https://")
                    for i in soup.find_all("img",{"src":lambda x:x and "avatars.mds.yandex.net/get-mpic" in x and "1hq" in x})]

    el = soup.find("div", {"data-apiary-widget-id": "/content/reactProductSummary"})
    if soup.find("ul", {"class": "n-product-spec-list"}) is not None:
        info["specifications"] = [li.text for li in
                                  soup.find("ul", {"class": "n-product-spec-list"}).find_all("li")]
    else:
        info["specifications"] = [li.text for li in el.find_all("li")]
    info["specifications"] = list(filter(lambda x: x!='',info["specifications"]))
    return info

In [36]:
re.search(r"\d+",soup.find("a", {"href": lambda x: x and "/reviews" in x}).text.replace(" ","")).group(0)

'351'

In [52]:
driver.refresh()

In [ ]:
%%time 

def main(urls,proxies):
    """
    Запускаем программу
    """
    queue = Queue()
    
    # Даем очереди нужные нам ссылки для скачивания
    for url in urls:
        queue.put(url)
 
    with open("cookies.pickle","rb") as f:
        cookies = pickle.load(f)
    
    # Запускаем потом и очередь
    threads = []
    for i in range(len(proxies)):
        t = Downloader(queue,i,cookies,proxies[i])
        t.start()
        threads.append(t)
    
    # Ждем завершения работы очереди
    for t in threads:
        t.join()

proxies=[
    {"host":"s1.neafiol.site","port":1080},
    {},
    {},
]
urls = [
    "https://market.yandex.ru/product--smartfon-samsung-galaxy-a50-64gb/394273081",
    "https://market.yandex.ru/product--smartfon-samsung-galaxy-a51-64gb/650874000",
    "https://market.yandex.ru/product--smartfon-google-pixel-3a-64gb/455014053"
]
urls=urls
print(len(urls)) 
main(urls,proxies)